In [7]:
try:
    from git_nomad import pull_nomad
except:
    import sys; sys.path.insert(0, '..')
    from git_nomad import pull_nomad
    
import socket
if socket.gethostname() == 'SPA-DESKTOP':
    remote_path = r"D:\Work_TU\Documents\nomad"
else:
    remote_path = r"C:\Users\martijnsparnaa\Eclipse_Python_NOMAD\nomad_trb_paper"
repo_path, repo = pull_nomad(remote_path, 'master')
print(repo_path.resolve())

import os
import time
os.chdir(repo_path)
headcommit = repo.head.commit
committed_date = time.strftime("%a, %d %b %Y %H:%M", time.localtime(headcommit.committed_date))
print(f'{headcommit}: {committed_date} - {headcommit.message}')

C:\Users\martijnsparnaa\Jupyterlab_notebooks\trb_analysis\nomad
1b0328f0ab9b5877b816e7c3658fa4b60db98f0a: Thu, 15 Jul 2021 22:34 - Merge branch 'master' of C:/Users/martijnsparnaa/Eclipse_Python_NOMAD/nomad_trb_paper



In [23]:
import os

print(os.getcwd())

C:\Users\martijnsparnaa\Jupyterlab_notebooks\trb_analysis\nomad


In [48]:
from _collections import defaultdict
from pathlib import Path

import json
import numpy as np

from contact_analysis import STAFF_CUSTOMER, CUSTOMER_CUSTOMER, INTERACTION_TYPES

connection_filename = Path(r'C:\Users\martijnsparnaa\surfdrive\PhD\Activity scheduler\Scenarios\paper_scenarios\base_output_verif\trb_paper_base_20210715234357_015.conn')

with open(connection_filename, 'r') as f:
    loaded_data = json.load(f)
    
# Node (ID, size=size, group=groupID)
# Edge (Node_ID_0, Node_ID_1, weight=weight)
    
dict_ID_2_ID_tuple = loaded_data['ID2IDtuple']
ID_2_group_tuple = loaded_data['ID2groupID']

cut_off_distances = (1.0, 1.5, 2.0)
weigths_per_connection = {cut_off_distance:{} for cut_off_distance in cut_off_distances}
weigths_per_agent = {cut_off_distance:defaultdict(float) for cut_off_distance in cut_off_distances}

ID_2_ID_tuple = defaultdict(list)

max_weight_agent = 0
max_weight_conn = 0

for dict_ID, ID_tuple in dict_ID_2_ID_tuple.items():
    ID_0 = ID_tuple[0]
    ID_1 = ID_tuple[1]
    ID_2_ID_tuple[ID_0].append(ID_tuple)
    ID_2_ID_tuple[ID_1].append(ID_tuple)
    connection_data = loaded_data[dict_ID]
    valueArray = np.array(connection_data)
    for cut_off_distance in cut_off_distances:        
        connectionWeight = float(np.sum(valueArray[:,1] <= cut_off_distance))
        if connectionWeight < 100:
            continue   
        if connectionWeight > max_weight_conn:
            max_weight_conn = connectionWeight
            
        weigths_per_connection[cut_off_distance][tuple(ID_tuple)] = connectionWeight
        weigths_per_agent[cut_off_distance][ID_0] += connectionWeight
        weigths_per_agent[cut_off_distance][ID_1] += connectionWeight
        
        if weigths_per_agent[cut_off_distance][ID_0] > max_weight_agent:
            max_weight_agent = weigths_per_agent[cut_off_distance][ID_0]
        if weigths_per_agent[cut_off_distance][ID_1] > max_weight_agent:
            max_weight_agent = weigths_per_agent[cut_off_distance][ID_1]

from pyvis import network as net
import networkx as nx
        
cut_off_distanc = cut_off_distances[1]

weight_factor_agent = 100/max_weight_agent
weight_factor_conn = 1/max_weight_conn

nx_graph = nx.Graph()
for agent_ID, weight in weigths_per_agent[cut_off_distance].items():
    nx_graph.add_node(agent_ID, size=min(weight*weight_factor_agent, 20), group=ID_2_group_tuple[str(agent_ID)])

for ID_tuple, weight in weigths_per_connection[cut_off_distance].items():
    nx_graph.add_edge(ID_tuple[0], ID_tuple[1], weight=weight*weight_factor_conn, value=weight*weight_factor_conn)

import os
#print(os.getcwd())
#os.chdir('..')
print(os.getcwd())
    
nt = net.Network(height=1000, width=1200, notebook=True)
nt.from_nx(nx_graph)
nt.show_buttons(filter_=['physics'])
nt.show('ex.html')


C:\Users\martijnsparnaa\Jupyterlab_notebooks\trb_analysis
